## Library

In [28]:
import requests
import random
import pandas as pd
import time
import math
from copy import deepcopy
from statistics import mean

## Notebook

#### Requisição

In [29]:
def request() -> list:   
    res = requests.get('http://localhost:3000/api/ep1')
    return res.json()

#### Tratamento de Dados

In [30]:
def data_treatment(json:list):
    for i in range(len(json)):
        for key in json[i]:
            if ('prod' in key):
                if (type(json[i][key])!=dict):
                    qnt = json[i][key]
                    if (type(qnt)==str):
                        qnt = float(qnt)                  
                    if (qnt<0):
                        qnt = 0
                    if (key in [product for product in json[i] if ('prod' in product and (int(product[5]) < 9 and len(product) < 7))]):
                        json[i][key] = {'qnt':[round(qnt)], 'price':None}
                    else:
                        json[i][key] = {'qnt':[qnt], 'price':None}
        try:
            json[i]['date'] = time.ctime(json[i]['date'])
        except:
            continue
    return json

#### Gerar valores aleatórios

In [31]:
def generate_random_price(n_min=1.0, n_max=100.0) -> float:
    return round(random.uniform(n_min, n_max), 2)

#### Precificação

##### Função logistica -> A mesma postada no doc pelo professor

In [32]:
def logistic_function(v: float) -> float:
    return 0.5 + (1 / (1 + math.e ** -v))

##### Função para calcular a variação -> A mesma postada no doc pelo professor

In [33]:
def calc_qnt_variation(last_week: list, current_week: list) -> float:
    return mean(current_week) - mean(last_week) / mean(last_week) if (mean(last_week) != 0) else 0

##### Função que calcula o preço que será utilizado na semana seguinte

In [34]:
def calc_new_price(product_db: dict, sales_week: dict) -> dict:
    for product in product_db:
        if (product_db[product]['qnt'] != None):
            if (product not in sales_week.keys()):
                sales_week[product] = [0]
            product_db[product]['next_week_price'] = logistic_function(calc_qnt_variation(product_db[product]['qnt'], sales_week[product])) * product_db[product]['price']
        product_db[product]['qnt'] = sales_week[product]               
    return product_db

##### Função que atualiza o banco de dados (onde consta a quantidade de vendas da última semana e o preço dos produtos) e aplica o preço nos produtos do JSON

In [35]:
def product_pricing(json: list, product_db: dict) -> tuple:
    sales_week = {}

    for transaction in json:
        for key in transaction:
            if ('prod' in key):
                if (key not in product_db.keys()):
                    transaction[key]['price'] = generate_random_price()
                    sales_week[key] = deepcopy(transaction[key]['qnt'])
                    product_db[key] = deepcopy(transaction[key])
                    product_db[key]['qnt'] = None
                    product_db[key]['next_week_price'] = product_db[key]['price']              
                else:
                    transaction[key]['price'] = deepcopy(product_db[key]['next_week_price'])
                    product_db[key]['price'] = product_db[key]['next_week_price']
                    if (key not in sales_week.keys()):
                        sales_week[key] = deepcopy(transaction[key]['qnt'])
                    else:
                        sales_week[key] += deepcopy(transaction[key]['qnt'])

    product_db = calc_new_price(product_db, sales_week)

    return json, product_db

##### Função que formata o DataFrame

In [36]:
def df_to_all_df(json, all_df, week):
    for transacao in json:
        json_to_df = deepcopy(transacao)
        date = json_to_df['date']
        id = json_to_df['id']
        del json_to_df['date']
        del json_to_df['id']
        df = pd.DataFrame(json_to_df).T
        df['id'] = id
        df['date'] = date
        df['month'] = pd.to_datetime(df['date']).dt.to_period('M')
        df['week'] = week
        df = pd.concat([all_df, df])
        # df=df.set_index(['date','id'])

    return df

In [37]:
def format_all_df(all_df:pd.DataFrame):
    all_df=all_df.reset_index().rename(columns={'index':'product'}).set_index('id').reset_index()
    all_df['qnt'] = [float(item[0]) for item in all_df['qnt']]
    return all_df

#### Main

In [48]:
product_database = {}
weeks = 8
all_df = pd.DataFrame([])

for week in range(weeks):
    json = data_treatment(request())
    json, product_database = product_pricing(json, product_database)

    all_df = df_to_all_df(json=json, all_df=all_df, week=week)
        
all_df=format_all_df(all_df=all_df)
all_df.to_csv('allsales.csv')

In [49]:
all_df.head()

,id,product,qnt,price,date,month,week
0,lafu6eipcgf0izakn6m,prod_0,26.0,80.92,Thu Jan 7 04:59:03 2021,2021-01,0
1,lafu6eipcgf0izakn6m,prod_1,40.0,40.04,Thu Jan 7 04:59:03 2021,2021-01,0
2,lafu6eipcgf0izakn6m,prod_2,49.0,4.82,Thu Jan 7 04:59:03 2021,2021-01,0
3,lafu6eipcgf0izakn6m,prod_4,7.0,81.63,Thu Jan 7 04:59:03 2021,2021-01,0
4,lafu6eipcgf0izakn6m,prod_5,40.0,30.79,Thu Jan 7 04:59:03 2021,2021-01,0


In [50]:
all_df.tail()

,id,product,qnt,price,date,month,week
60,lafu6eu8s8qen147pnn,prod_3,37.000000,99.554062,Thu Feb 25 08:32:02 2021,2021-02,7
61,lafu6eu8s8qen147pnn,prod_4,18.000000,929.816712,Thu Feb 25 08:32:02 2021,2021-02,7
62,lafu6eu8s8qen147pnn,prod_8,37.000000,52.396875,Thu Feb 25 08:32:02 2021,2021-02,7
63,lafu6eu8s8qen147pnn,prod_11,727.483206,133.384219,Thu Feb 25 08:32:02 2021,2021-02,7
64,lafu6eu8s8qen147pnn,prod_15,619.698185,809.987344,Thu Feb 25 08:32:02 2021,2021-02,7


In [51]:
all_df['month'].unique()

<PeriodArray>
['2021-01', '2021-02']
Length: 2, dtype: period[M]